<a href="https://colab.research.google.com/github/PAYAL980/Tensor-Network-for-Machine-Learning/blob/main/Final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensornetwork

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.3/364.3 kB 7.1 MB/s eta 0:00:00


In [2]:
# Importing libraries

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Import tensornetwork
import tensornetwork as tn
# Set the backend to tesorflow
# (default is numpy)
tn.set_default_backend("tensorflow")
import time

## Final model


In [102]:
## final model

class model(tf.keras.layers.Layer):
  def __init__(self):
    super(model, self).__init__()

    self.vars = []
    L = 784


    # first tensor has only two indices
    self.vars.append(tf.Variable(np.float32(np.eye(2,2)+np.random.normal(0,1e-2,size=(2, 2))), trainable=True))

        # all middle tensors have 3 indices
    for i in range(L-2):
      self.vars.append(tf.Variable(np.float32([np.eye(2,2) for iz in range(2)]+np.random.normal(0,1e-2,size=(2,2,2))), trainable=True))

        # last tensor will have output, so it will hold 3 indices
    self.vars.append(tf.Variable(np.float32([np.eye(2,10) for iz in range(2)]+np.random.normal(0,1e-2,size=(2, 2, 10))), trainable=True))

    # bias
    self.bias = tf.Variable(tf.zeros(shape=(10)),
                          name="bias", trainable=True)

  def call(self, inputs):

    def f(input_feature_map, vars, bias):

      L = 784

      # Function to connect the mps

      def connect_mps(mps, num_inds):
        for iz in range(len(mps)):

          if iz==0:
            mps[iz][num_inds-2]^mps[iz+1][num_inds-2]

          if iz!=0 and iz!=len(mps)-1:
            mps[iz][num_inds-1]^mps[iz+1][num_inds-2]

        return mps

      # Function for contracting the nodes

      def contract_data(a,b):
        return (a@b).tensor

      input_tensors = []
      mps = []
      cntrct = []

      # creating data tensor and mps nodes and connecting to each other
      for iz in range(L):
        input_tensors.append(tn.Node(input_feature_map[iz], name = f'I{iz}'))
        mps.append(tn.Node(vars[iz], name=f'M{iz}'))

        input_tensors[iz][0]^mps[iz][0]
        cntrct.append(tn.Node(contract_data(input_tensors[iz], mps[iz]), name=f'C{iz}'))

      # connecting the mps

      mps_new = connect_mps(cntrct, num_inds=2)


      # Now let's contract the nodes
      while len(mps_new)!=3 and len(mps_new)!=2 :

        if len(mps_new)%2==1:
          mps_new = [tn.Node(contract_data(mps_new[iz], mps_new[iz+1]), name = f'R{iz}') for iz in range(0, len(mps_new)-1, 2)] + [tn.Node((mps_new[len(mps_new)-1]), name='last')]

        else:
          mps_new = [tn.Node(contract_data(mps_new[iz], mps_new[iz+1]), name = f'R{iz}') for iz in range(0, len(mps_new)-2, 2)]+[tn.Node((mps_new[len(mps_new)-2])), tn.Node((mps_new[len(mps_new)-1]),name='last')]

        mps_new = connect_mps(mps_new, num_inds=2)


      if len(mps_new)==2:
        mps_final = mps_new[0] @ mps_new[1]

      if len(mps_new)==3:
        mps_final = mps_new[0] @ mps_new[1]
        mps_final = mps_final @ mps_new[2]


      d = mps_final.get_tensor() + bias

      return d

    # To deal with a batch of items, we can use the tf.vectorized_map function

    result = tf.vectorized_map(
        lambda vec: f(vec, self.vars, self.bias), inputs)

    return tf.nn.softmax(result)


In [103]:
%%time

model_1 = tf.keras.Sequential(
    [
    tf.keras.Input(shape=(784, 2,), batch_size=50),
    model(),
    ])
model_1.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_15 (model)            (50, 10)                  6310      
                                                                 
Total params: 6,310
Trainable params: 6,310
Non-trainable params: 0
_________________________________________________________________
CPU times: user 28.3 s, sys: 139 ms, total: 28.4 s
Wall time: 28.6 s
